
# Fixed-Point Iteration & Newton's Method — From Scratch

This notebook cleans up and extends your scripts into a **teach-by-doing** TP:
- **Logistic fixed-point** iteration: \( f_a(x) = a\,x(1-x) \)
  - theory (contraction, fixed points), trajectories, convergence regions
  - error vs tolerance, effect of parameter \(a\)
- **Root-finding** for \( x - \sin x = \tfrac{3\pi}{2} \):
  - **Newton's method** vs **fixed-point** \(x=h(x)\) with \(h(x)=\sin x + \tfrac{3\pi}{2}\)
  - convergence comparison and error scaling

Rules we follow:
- One plot per figure; Matplotlib only.
- Guardrails for divergences (max iterations, safety checks).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm

%matplotlib inline



## Part I — Fixed-Point Iteration on the Logistic Map

We consider \( f_a(x) = a\,x(1-x) \) on \([0,1]\). Fixed points solve \(x=f_a(x)\):
- \(x_0^\*=0\) (always a fixed point)
- \(x_1^\*=(a-1)/a\) (exists for \(a\neq 0\))

Local convergence (Banach): if \(|f'(x^\*)|<1\), fixed-point iteration converges near \(x^\*\).  
Here \( f'_a(x) = a(1-2x) \). At \(x_1^\*\), \( f'_a(x_1^\*) = 2-a \).  
So \( |2-a|<1 \iff 1<a<3 \) → the nontrivial fixed point is **attractive** for \(1<a<3\).


In [ ]:
def f_logistic(x, a):
    return a*x*(1.0 - x)

def logistic_fixed_points(a):
    if a == 0:
        return [0.0]
    return [0.0, (a - 1.0)/a]

def fixed_point_iterate(f, x0, tol=1e-8, maxit=10000, *, args=()):
    """Generic fixed-point iteration x_{k+1}=f(x_k).
    Returns trajectory array and stop flag.
    """
    x = float(x0)
    traj = [x]
    for k in range(maxit):
        xn = f(x, *args)
        traj.append(xn)
        if abs(xn - x) < tol:
            return np.array(traj), True
        x = xn
    return np.array(traj), False



### Trajectories for different \(a\)

We start from \(x_0=0.9\) and iterate for a few values of \(a\).


In [ ]:
a_list = [0.8, 1.6, 2.0, 2.9]
x0 = 0.9
tol = 1e-6

for a in a_list:
    traj, ok = fixed_point_iterate(f_logistic, x0, tol=tol, maxit=50_000, args=(a,))
    xs = np.arange(len(traj))
    print(f"a={a:>4}: converged={ok}, last x={traj[-1]:.8f}, steps={len(traj)-1}")
    plt.figure()
    plt.plot(xs, traj, '.-')
    plt.axhline(0.0, color='k', lw=0.7)
    if a != 0:
        plt.axhline((a-1)/a, color='r', lw=0.7, label='(a-1)/a')
        plt.legend()
    plt.xlabel('iteration k'); plt.ylabel('x_k')
    plt.title(f'Logistic fixed-point trajectory (a={a}, x0={x0})')
    plt.show()



### Error vs tolerance (observed)

For \(1<a<3\), the attractive fixed point is \(x^\*=(a-1)/a\).  
We measure \(|x_{\text{final}}-x^\*|\) as a function of the stopping tolerance.


In [ ]:
def terminal_error_vs_tol(a, x0=0.9, tols=None):
    if tols is None:
        tols = np.logspace(-6, -1, 10)
    xstar = (a-1)/a if a != 0 else 0.0
    errs = []
    for tol in tols:
        traj, ok = fixed_point_iterate(f_logistic, x0, tol=tol, maxit=100_000, args=(a,))
        errs.append(abs(traj[-1]-xstar))
    return tols, np.array(errs)

a = 2.0
tols, errs = terminal_error_vs_tol(a)
plt.figure()
plt.loglog(1/tols, errs, 'o-')
plt.xlabel('1/tolerance (proxy for tighter stopping)')
plt.ylabel('|x_final - x*|')
plt.title(f'Error vs tolerance (a={a})')
plt.grid(True, which='both')
plt.show()



### Graphical iteration (cobweb)

Cobweb plots illustrate convergence vs divergence depending on \(a\).


In [ ]:
def cobweb_plot(f, a, x0=0.9, n=50, xmin=0, xmax=1):
    x = np.linspace(xmin, xmax, 400)
    y = f(x, a)
    plt.figure()
    plt.plot(x, y, 'b', label='f(x)')
    plt.plot(x, x, 'k--', lw=1, label='y=x')
    # cobweb
    xk = x0
    for _ in range(n):
        yk = f(xk, a)
        plt.plot([xk, xk], [xk, yk], 'r-', lw=0.8)
        plt.plot([xk, yk], [yk, yk], 'r-', lw=0.8)
        xk = yk
        if xk < xmin or xk > xmax:
            break
    plt.xlim(xmin, xmax); plt.ylim(xmin, xmax)
    plt.xlabel('x'); plt.ylabel('y')
    plt.title(f'Cobweb plot (a={a}, x0={x0})')
    plt.legend(); plt.show()

for a in [0.8, 1.6, 2.0, 2.9]:
    cobweb_plot(f_logistic, a, x0=0.9, n=50, xmin=0, xmax=1)



## Part II — Root-Finding: \( x - \sin x = \tfrac{3\pi}{2} \)

Define \( g(x)= x - \sin x - \tfrac{3\pi}{2} \). We seek \(g(x)=0\).  
Two approaches:
- **Newton**: \( x_{k+1}=x_k - \frac{g(x_k)}{g'(x_k)} \) with \( g'(x)=1-\cos x \).
- **Fixed-point**: \( x=h(x)=\sin x + \tfrac{3\pi}{2} \) (works if \(h\) is contractive near the solution).  
  Here \(h'(x)=\cos x\), and near the solution (around \(3\\pi/2\)) \(|\cos x|\approx0\), so it can converge.


In [ ]:
def g(x):
    return x - np.sin(x) - 3*np.pi/2

def gprime(x):
    return 1 - np.cos(x)

def newton(g, gprime, x0, maxit=100, tol=1e-12):
    x = float(x0)
    for k in range(maxit):
        gp = gprime(x)
        if abs(gp) < 1e-14:
            break
        x_new = x - g(x)/gp
        if abs(x_new - x) < tol:
            return x_new, True, k+1
        x = x_new
    return x, False, maxit

def fixed_point_scalar(h, x0, maxit=10_000, tol=1e-12):
    x = float(x0)
    for k in range(maxit):
        x_new = h(x)
        if abs(x_new - x) < tol:
            return x_new, True, k+1
        x = x_new
    return x, False, maxit

h = lambda x: np.sin(x) + 3*np.pi/2

# Compare starting from same x0
x0 = 1.0
xn, okn, kn = newton(g, gprime, x0)
xf, okf, kf = fixed_point_scalar(h, x0)

print(f"Newton: x={xn:.12f}, converged={okn}, iters={kn}")
print(f"Fixed-pt: x={xf:.12f}, converged={okf}, iters={kf}")

# The true solution is near 3*pi/2 ≈ 4.71238898
x_true = 4.71238898038469
print("|Newton - true|=", abs(xn - x_true))
print("|Fixed - true| =", abs(xf - x_true))



### Error vs tolerance for the fixed-point version

We compare the fixed-point solution against Newton's method output (as a proxy), while tightening the tolerance.


In [ ]:
def fixed_point_error_vs_tol(h, x0, ref, tols=None):
    if tols is None:
        tols = np.logspace(-1, -6, 12)
    errs = []
    for tol in tols:
        x, ok, k = fixed_point_scalar(h, x0, tol=tol, maxit=100_000)
        errs.append(abs(x - ref))
    return tols, np.array(errs)

# use Newton's solution as reference since it converges very fast
x_ref, _, _ = newton(g, gprime, 1.0)
tols, errs = fixed_point_error_vs_tol(h, 1.0, x_ref)

plt.figure()
plt.loglog(1/tols, errs, 'o-')
plt.xlabel('1/tolerance'); plt.ylabel('|x_fixed - x_ref(Newton)|')
plt.title('Fixed-point accuracy vs stopping tolerance')
plt.grid(True, which='both')
plt.show()
